In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
tree_df = pd.DataFrame(data = {"Daughter_1": [2, 1, 4, 11, 10, 7, 13],
                               "Branch_Length_1": [2.7, 1.8, 1.1, 1.9, 1.5, 4.7, 2.2],
                               "Daughter_2": [3, 9, 5, 6, 12, 8, 14],
                               "Branch_Length_2": [2.7, 4.5, 1.1, 3, 3, 4.7, 3.5]},
                       index = [9, 10, 11, 12, 13, 14, 15])

display(tree_df)

,Daughter_1,Branch_Length_1,Daughter_2,Branch_Length_2
9,2,2.7,3,2.7
10,1,1.8,9,4.5
11,4,1.1,5,1.1
12,11,1.9,6,3.0
13,10,1.5,12,3.0
14,7,4.7,8,4.7
15,13,2.2,14,3.5


In [75]:
class TreeInference():
    """
    tree_data: dataframe of nodes, daughters and branch length
    mu: mutation rate
    leaf_values: bases for leaf nodes
    """
    def __init__(self, tree_data, mu, leaf_values):
        # Set parameters
        self.tree_data = tree_data
        self.mu = mu
        self.leaf_values = leaf_values
        
        # Find IDs of internal nodes, all nodes and leaf nodes
        self.internal_node_ids = np.array(tree_data.index)
        self.all_node_ids = np.unique(np.concatenate((np.array(tree_data.index),
                                                      np.array(tree_data["Daughter_1"]),
                                                      np.array(tree_data["Daughter_2"]))))
        self.leaf_node_ids = list(set(self.all_node_ids) - set(self.internal_node_ids))
        
        # Record branch lengths to leaves
        self.root_leaf_dist = {}
        
        # Count internal nodes and leaves for each row in tree_data
        self.treeNodeCounts()
        
        # Get indexes for Q and A matricies for states, e.g. A is row 0, C is row 2...
        states = np.unique(list(leaf_values.values()))
        self.matrix_idx = dict(zip(states, range(len(states))))
        
        # Create Q matrix for bases A, C, T, G
        self.Q = self.MatrixQ()
        # Create empty matrix of likelihood for each node
        self.L = np.full((len(self.all_node_ids), len(self.matrix_idx)), None)
        
    # For each internal node, calculate the number of internal nodes and leaves below
    def treeNodeCounts(self):
        n_nodes = np.array([])
        n_leaves = np.array([])
        max_leaves = 0

        # Set the number of internal nodes and leaves for each internal node row
        for n in self.internal_node_ids:
            num_nodes, num_leaves, leaf_dist = self.checkNode(self.tree_data,
                                                              n,
                                                              self.internal_node_ids)
            n_nodes = np.append(n_nodes, num_nodes)
            n_leaves = np.append(n_leaves, num_leaves)

            # Check if found root node
            if len(leaf_dist) > max_leaves:
                # Record branch lengths to leaves
                self.root_leaf_dist = leaf_dist
                max_leaves = len(leaf_dist)

        # Add new information to dataframe
        self.tree_data["N_Internal"] = n_nodes
        self.tree_data["N_Leaves"] = n_leaves
                
    # Recursively find the number of internal nodes and leaves given a node ID
    def checkNode(self, tree_data, node_id, node_array):
        # Record the number of internal nodes and leaves below
        num_nodes = 0
        num_leaves = 0
        # Record branch lengths of leaves below
        leaf_lengths = {}

        # Find the left and right daughter nodes
        daughter_1 = self.tree_data.loc[node_id]["Daughter_1"]
        daughter_2 = self.tree_data.loc[node_id]["Daughter_2"]

        # Check if found an internal node to the left
        if daughter_1 in node_array:
            # Recursively check nodes below
            d1_nodes, d1_leaves, d1_leaf_lengths = self.checkNode(self.tree_data,
                                                                  daughter_1, node_array)
            num_nodes += d1_nodes + 1
            num_leaves += d1_leaves

            # Increment branch lengths of leaves
            for l in d1_leaf_lengths.keys():
                leaf_lengths[l] = round(d1_leaf_lengths[l] + tree_data.loc[node_id]["Branch_Length_1"], 1)

        # Otherwise found a left leaf node
        else:
            num_leaves += 1
            leaf_lengths[daughter_1] = self.tree_data.loc[node_id]["Branch_Length_1"]

        # Check if found an internal node to the right
        if daughter_2 in node_array:
            d2_nodes, d2_leaves, d2_leaf_lengths = self.checkNode(self.tree_data,
                                                                  daughter_2,
                                                                  node_array)
            num_nodes += d2_nodes + 1
            num_leaves += d2_leaves

            for l in d2_leaf_lengths.keys():
                leaf_lengths[l] = round(d2_leaf_lengths[l] + self.tree_data.loc[node_id]["Branch_Length_2"], 1)

        # Otherwise found a right leaf node
        else:
            num_leaves += 1
            leaf_lengths[daughter_2] = self.tree_data.loc[node_id]["Branch_Length_2"]

        return num_nodes, num_leaves, leaf_lengths
    
    # Jukes-Cantor model Q matrix
    def MatrixQ(self):
        # Create matrix with all off-diagonal rates in Q as the same value
        Q = np.full((len(self.matrix_idx), len(self.matrix_idx)),
                    self.mu/(len(self.matrix_idx) - 1))
        np.fill_diagonal(Q, 1 - self.mu)
        return Q
    
    # Create transition matrix for time t
    def TransitionMatrixA(self, t, mu):
        A = np.full((4, 4), (1/4) * (1 - math.exp(-4 * mu * t)))
        np.fill_diagonal(A, (1/4) * (1 + 3 * math.exp(-4 * mu * t)))
        return A
    
    def nodeLikelihood(self, node_i, daughter_1, daughter_2):    
        # Get branch length times for child nodes
        t1 = self.tree_data.loc[node_i]["Branch_Length_1"]
        t2 = self.tree_data.loc[node_i]["Branch_Length_2"]
        
        # Create transition matrix for left and right child for t1 and t2
        A1 = self.TransitionMatrixA(t1, self.mu)
        A2 = self.TransitionMatrixA(t2, self.mu)
        
        # Get conditional likelihood of child nodes
        d1_likelihood = self.L[int(daughter_1) - 1]
        d2_likelihood = self.L[int(daughter_2) - 1]
        
        # Calculate the likelihood for each state for the node
        left_prob = np.sum(A1 * d1_likelihood, axis = 1)
        right_prob = np.sum(A2 * d2_likelihood, axis = 1)
        self.L[node_i - 1] = np.multiply(left_prob, right_prob)
    
    def calculateLikelihood(self):
        # Set probability of leaf states
        for l in self.leaf_node_ids:
            # Zero for non-matching base
            self.L[l - 1] = 0
            # One for matching base
            self.L[l - 1, self.matrix_idx[self.leaf_values[l]]] = 1
        
        for i in self.internal_node_ids:
            print("NODE " + str(i))
            # Check if probabilities not been set
            if self.L[i - 1, 0] is None:
                # Find the left and right daughter nodes
                daughter_1 = self.tree_data.loc[i]["Daughter_1"]
                daughter_2 = self.tree_data.loc[i]["Daughter_2"]

                if self.L[int(daughter_1) - 1, 0] is None:
                    print("None")
                else:
                    print("leaf")
                    
                # Get branch length times for child nodes
                t1 = self.tree_data.loc[i]["Branch_Length_1"]
                t2 = self.tree_data.loc[i]["Branch_Length_2"]
                    
                # Create transition matrix for left and right child for t1 and t2
                A1 = self.TransitionMatrixA(t1, self.mu)
                A2 = self.TransitionMatrixA(t2, self.mu)
                    
                # Get conditional likelihood of child nodes
                d1_likelihood = self.L[int(daughter_1) - 1]
                d2_likelihood = self.L[int(daughter_2) - 1]
                
                # Calculate the likelihood for each state for the node
                left_prob = np.sum(A1 * d1_likelihood, axis = 1)
                right_prob = np.sum(A2 * d2_likelihood, axis = 1)
                self.L[i - 1] = np.multiply(left_prob, right_prob)

In [76]:
inference = TreeInference(tree_df, mu = 0.01,
                          leaf_values = {1: "A", 2: "G", 3: "T", 4: "G", 5: "G", 6: "G", 7: "C", 8: "G"})

display(inference.tree_data)

# print("Distance for each leaf from root:")
# print(inference.root_leaf_dist)

# print("\nQ matrix: \n" + str(inference.Q))

# print("\nP matrix: \n" + str(inference.P))

print(inference.matrix_idx)

inference.calculateLikelihood()


inference.L


,Daughter_1,Branch_Length_1,Daughter_2,Branch_Length_2,N_Internal,N_Leaves
9,2,2.7,3,2.7,0.0,2.0
10,1,1.8,9,4.5,1.0,3.0
11,4,1.1,5,1.1,0.0,2.0
12,11,1.9,6,3.0,1.0,3.0
13,10,1.5,12,3.0,4.0,6.0
14,7,4.7,8,4.7,0.0,2.0
15,13,2.2,14,3.5,6.0,8.0


{'A': 0, 'C': 1, 'G': 2, 'T': 3}
NODE 9
leaf
NODE 10
leaf
NODE 11
leaf
NODE 12
leaf
NODE 13
leaf
NODE 14
leaf
NODE 15
leaf


array([[1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0.0006550068132881193, 0.0006550068132881193,
        0.02362808045252692, 0.02362808045252692],
       [0.002414468777482083, 4.4237605906193515e-05,
        0.00037749347391742853, 0.00037749347391742853],
       [0.00011581011107701711, 0.00011581011107701711,
        0.9364732272092632, 0.00011581011107701711],
       [0.0004875809010589545, 0.0004875809010589545, 0.8100153613472278,
        0.0004875809010589546],
       [5.4240159633912015e-05, 2.067307870154045e-06,
        0.000298247250522619, 9.40284088046066e-06],
       [0.0018358074110587364, 0.037338900958653644,
        0.037338900958653644, 0.0018358074110587364],
       [2.382249046524016e-07, 3.347202342100392e-07,
        9.833142778092193e-06, 6.762358759681414e-08]], dtype=object)